In [60]:
import pandas as pd
import numpy as np
import re

In [61]:
yearsToCheck = list((np.arange(2000, 2020, 1)).astype(str))
developedCountries = ['Australia','Japan','Sweden','Switzerland','USA']
developingCountries = ['Brazil','Egypt','Greece','India','South Africa']
underdevelopedCountries = ['Afghanistan','Bangladesh','Cambodia','Ethiopia','Senegal']

In [62]:
populationCsvPath = 'data/raw_data/pop.csv'
readPopCsv = pd.read_csv(populationCsvPath)
populationDataFrame = (pd.DataFrame(readPopCsv).set_index('country'))[yearsToCheck]

developed_populationDataFrame = populationDataFrame.loc[developedCountries]
developing_populationDataFrame = populationDataFrame.loc[developingCountries]
underdeveloped_populationDataFrame = populationDataFrame.loc[underdevelopedCountries]

df = [developed_populationDataFrame, underdeveloped_populationDataFrame, developing_populationDataFrame]

for j in df:
    for i in yearsToCheck:
        for index in range(len(j[i])):
            try:
                if (j[i][index][-1] == 'M') & ('.' not in j[i][index]):
                    j[i][index] = j[i][index].replace('M', '000000')
                elif (j[i][index][-1] == 'B') & ('.' not in j[i][index]):
                    j[i][index] = j[i][index].replace('B', '000000000')
                elif (j[i][index][-1] == 'M') & (j[i][index][-3] == '.'):
                    j[i][index] = j[i][index].replace('.', '').replace('M', '00000')
                elif (j[i][index][-1] == 'B') & (j[i][index][-3] == '.'):
                    j[i][index] = j[i][index].replace('.', '').replace('B', '00000000')
                elif (j[i][index][-1] == 'M') & (j[i][index][-4] == '.'):
                    j[i][index] = j[i][index].replace('.', '').replace('M', '0000')
                elif (j[i][index][-1] == 'B') & (j[i][index][-4] == '.'):
                    j[i][index] = j[i][index].replace('.', '').replace('B', '0000000')

            except:
                print(j[i][index])



developed_populationDataFrame.to_csv('data/clean_data/population/csv/developed_population.csv')
developing_populationDataFrame.to_csv('data/clean_data/population/csv/developing_population.csv')
underdeveloped_populationDataFrame.to_csv('data/clean_data/population/csv/underveloped_population.csv')

developed_populationDataFrame.to_json('data/clean_data/population/json/developed_population.json', indent = 1)
developing_populationDataFrame.to_json('data/clean_data/population/json/developing_population.json', indent = 1)
underdeveloped_populationDataFrame.to_json('data/clean_data/population/json/underveloped_population.json', indent = 1)

In [110]:
gdpUSDPath = 'data/raw_data/total_gdp_us_inflation_adjusted.csv'
readGdpUSDCsv = pd.read_csv(gdpUSDPath, index_col = 'country')
GdpDataFrame = pd.DataFrame(readGdpUSDCsv)[yearsToCheck]

developed_GdpDataFrame = GdpDataFrame.loc[developedCountries]
developing_GdpDataFrame = GdpDataFrame.loc[developingCountries]
underdeveloped_GdpDataFrame = GdpDataFrame.loc[underdevelopedCountries].fillna(0)

df = [developed_GdpDataFrame, developing_GdpDataFrame, underdeveloped_GdpDataFrame]

for each in df:
    for year in yearsToCheck:
        try:
            for index in range(len(each[year])):
                iteration = str(each[year][index])
                split_iteration = iteration.split('.')

                if 'B' in split_iteration[0]:
                    each[year][index] = iteration.replace('B', '000000000')
                elif 'TR' in split_iteration[0]:
                    each[year][index] = iteration.replace('TR', '000000000000')            
                try: 
                    if len(split_iteration[1]) == 2:
                        each[year][index] = iteration.replace('B', '00000000').replace('.', '')                    
                    elif 'TR' in split_iteration[1]:
                        if (len(split_iteration[1]) == 3):
                            each[year][index] = iteration.replace('TR', '00000000000').replace('.', '')                        
                        elif (len(split_iteration[1]) == 4):
                            each[year][index] = iteration.replace('TR', '0000000000').replace('.', '')
                    elif 'B' in split_iteration[1]:
                        if (len(split_iteration[1]) == 2):
                            each[year][index] = iteration.replace('B', '00000000').replace('.', '') 
                        elif (len(split_iteration[1]) == 3):
                            each[year][index] = iteration.replace('B', '0000000').replace('.', '')
                       
                except:
                    print(iteration)
        except:
            print(year)
                

developed_GdpDataFrame.to_csv('data/clean_data/gdpUSD/csv/developed_gdp.csv')
developing_GdpDataFrame.to_csv('data/clean_data/gdpUSD/csv/developing_gdp.csv')
underdeveloped_GdpDataFrame.to_csv('data/clean_data/gdpUSD/csv/underdeveloped_gdp.csv')

developed_GdpDataFrame.to_json('data/clean_data/gdpUSD/json/developed_gdp.json', indent = 1)
developing_GdpDataFrame.to_json('data/clean_data/gdpUSD/json/developing_gdp.json', indent = 1)
underdeveloped_GdpDataFrame.to_json('data/clean_data/gdpUSD/json/underdeveloped_gdp.json', indent = 1)

873B
365B
530B
890B
4TR
370B
538B
926B
4TR
378B
538B
954B
387B
538B
995B
404B
552B
415B
567B
435B
590B
16TR
450B
614B
448B
631B
428B
616B
16TR
454B
636B
468B
648B
465B
655B
17TR
471B
667B
483B
683B
505B
694B
516B
708B
529B
718B
539B
739B
550B
747B
180B
201B
801B
222B
186B
210B
839B
228B
190B
218B
871B
236B
197B
231B
940B
243B
205B
242B
254B
214B
244B
268B
228B
258B
283B
245B
266B
298B
262B
265B
307B
274B
254B
302B
288B
240B
312B
294B
215B
322B
300B
200B
329B
307B
195B
337B
316B
196B
342B
329B
196B
347B
344B
195B
349B
358B
197B
353B
377B
200B
359B
398B
204B
359B
0
0
100B
107B
114B
122B
129B
12B
136B
12B
144B
153B
163B
48B
173B
183B
195B
18B
209B
223B
239B
258B


In [66]:
avgDailyIncPpPath = 'data/raw_data/mincpcap_cppp.csv'
readvgDailyIncCsv = pd.read_csv(avgDailyIncPpPath)
avgDailyIncPpDataFrame = (pd.DataFrame(readvgDailyIncCsv).set_index('country'))[yearsToCheck]

developed_avgDailyIncPpDataFrame = avgDailyIncPpDataFrame.loc[developedCountries]
developing_avgDailyIncPpDataFrame = avgDailyIncPpDataFrame.loc[developingCountries]
underdeveloped_avgDailyIncPpDataFrame = avgDailyIncPpDataFrame.loc[underdevelopedCountries]

developed_avgDailyIncPpDataFrame.to_csv('data/clean_data/averageIncomePerPerson/csv/developed_average_daily_income.csv')
developing_avgDailyIncPpDataFrame.to_csv('data/clean_data/averageIncomePerPerson/csv/developing_average_daily_income.csv')
underdeveloped_avgDailyIncPpDataFrame.to_csv('data/clean_data/averageIncomePerPerson/csv/underdeveloped_average_daily_income.csv')

developed_avgDailyIncPpDataFrame.to_json('data/clean_data/averageIncomePerPerson/json/developed_average_daily_income.json', indent = 1)
developing_avgDailyIncPpDataFrame.to_json('data/clean_data/averageIncomePerPerson/json/developing_average_daily_income.json', indent = 1)
underdeveloped_avgDailyIncPpDataFrame.to_json('data/clean_data/averageIncomePerPerson/json/underdeveloped_average_daily_income.json', indent = 1)

In [67]:
# This is in tonnes
co2PerCapPath = 'data/raw_data/co2_pcap_cons.csv'
readco2PerCapCsv = pd.read_csv(co2PerCapPath)
co2PerCapDataFrame = (pd.DataFrame(readco2PerCapCsv).set_index('country'))[yearsToCheck]

developed_co2PerCapDataFrame = co2PerCapDataFrame.loc[developedCountries]
developing_co2PerCapDataFrame = co2PerCapDataFrame.loc[developingCountries]
underdeveloped_co2PerCapDataFrame = co2PerCapDataFrame.loc[underdevelopedCountries]

developed_co2PerCapDataFrame.to_csv('data/clean_data/co2PerCapitaConsumption/csv/developed_co2_per_capita.csv')
developing_co2PerCapDataFrame.to_csv('data/clean_data/co2PerCapitaConsumption/csv/developing_co2_per_capita.csv')
underdeveloped_co2PerCapDataFrame.to_csv('data/clean_data/co2PerCapitaConsumption/csv/underdeveloped_co2_per_capita.csv')

developed_co2PerCapDataFrame.to_json('data/clean_data/co2PerCapitaConsumption/json/developed_co2_per_capita.json', indent = 1)
developing_co2PerCapDataFrame.to_json('data/clean_data/co2PerCapitaConsumption/json/developing_co2_per_capita.json', indent = 1)
underdeveloped_co2PerCapDataFrame.to_json('data/clean_data/co2PerCapitaConsumption/json/underdeveloped_co2_per_capita.json', indent = 1)

In [68]:
# This is in million tonnes
co2TotalEmissionsPath = 'data/raw_data/co2_cons.csv'
readco2TotalEmissionCsv = pd.read_csv(co2TotalEmissionsPath)
co2TotalEmissionsDataFrame = (pd.DataFrame(readco2TotalEmissionCsv).set_index('country'))[yearsToCheck]

developed_co2TotalEmissionsDataFrame = co2TotalEmissionsDataFrame.loc[developedCountries]
developing_co2TotalEmissionsDataFrame = co2TotalEmissionsDataFrame.loc[developingCountries]
underdeveloped_co2TotalEmissionsDataFrame = co2TotalEmissionsDataFrame.loc[underdevelopedCountries]

developed_co2TotalEmissionsDataFrame.to_csv('data/clean_data/co2EmissionsTotal/csv/developed_co2_total_emissions.csv')
developing_co2TotalEmissionsDataFrame.to_csv('data/clean_data/co2EmissionsTotal/csv/developing_co2_total_emissions.csv')
underdeveloped_co2TotalEmissionsDataFrame.to_csv('data/clean_data/co2EmissionsTotal/csv/underdeveloped_co2_total_emissions.csv')

developed_co2TotalEmissionsDataFrame.to_json('data/clean_data/co2EmissionsTotal/json/developed_co2_total_emissions.json', indent = 1)
developing_co2TotalEmissionsDataFrame.to_json('data/clean_data/co2EmissionsTotal/json/developing_co2_total_emissions.json', indent = 1)
underdeveloped_co2TotalEmissionsDataFrame.to_json('data/clean_data/co2EmissionsTotal/json/underdeveloped_co2_total_emissions.json', indent = 1)